In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp, ttest_ind

In [2]:
!which python

/home/aalag/anaconda3/bin/python


In [2]:
!pip3 install ipykernel

  Using cached https://files.pythonhosted.org/packages/e9/ad/9101e0ab5e84dd117462bb3a1379d31728a849b6886458452e3d97dc6bba/ipykernel-5.5.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/56/a7/f4d3790ce7bb925d3ffe299244501a264f23ee7ec401914f7d788881ea31/jupyter_client-7.1.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/bc/9bd3b5c2b4774d5f33b2d544f1460be9df7df2fe42f352135381c347c69a/ipython_genutils-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/53/09/958a4802489d28b2484114ee6414c7502ef57de6f2dbc9095b718640060c/ipython-7.16.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/ab/872a23e29cec3cf2594af7e857f18b687ad21039c1f9b922fac5b9b142d5/traitlets-4.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/01/d1/8750ad20cbcefb499bb8b405e243f83c2c89f78d139e6f8c8d800640f554/tornado-6.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhost

In [3]:
def get_member_ratio(datadf, metrics, thre_cnt=1, skip=[], mode='ks'):
    sample2mem = []
    for idx, row in datadf.iterrows():
        sample2mem.append(np.sum([row[m] for m in (set(metrics) - set(skip))]))
    
    members_bool = np.asarray(sample2mem) >= thre_cnt

    if mode == 'ks': # ks distance
        _, pvalue = ks_2samp(members_bool,[1 for i in range(10000)] , mode='asymp')
    elif mode == 't': # majority
        if members_bool.mean() == 1:
            return 1, 1
        else:
            _, pvalue = ttest_ind(members_bool,[1 for i in range(len(members_bool))], equal_var = True, nan_policy='raise')
    elif mode == 'max':
        if max(members_bool) == 1:
            return 1, 1
        else:
            _, pvalue = ttest_ind(members_bool,[1 for i in range(len(members_bool))], equal_var = True, nan_policy='raise')
    elif mode == 'majority':
        if members_bool.sum() > members_bool.size // 2:
            return 1, 1
        else:
            _, pvalue = ttest_ind(members_bool,[1 for i in range(len(members_bool))], equal_var = True, nan_policy='raise')
         

    return np.sum(members_bool) / len(datadf), pvalue

In [7]:
!pwd

/della/scratch/gpfs/aalag/DataAuditingThesis


In [6]:
### Run EMA on the benchmark datasets, Table 2.b and Table 6
querydata = 'MNIST'
expt = "dropout0.1"
# expt=""
epoch = 50

# metric_main = ['correctness', 'confidence', 'entropy', 'modified entropy', 'scaled logits']
# metric_one = ['correctness', 'confidence']
# metric_two = ['correctness', 'entropy']
# metric_three = ['correctness', 'modified entropy']
# metric_four = ['correctness', 'scaled logits']

# all_metrics = [metric_main, metric_one, metric_two, metric_three, metric_four]

metrics = ['correctness']
all_metrics = [metrics]

klist = [0, 10, 20, 30, 40, 50]
mixup = False

EMA_res_table = pd.DataFrame(columns=['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'SVHN'], index=[f'k={100-k}' for k in klist])
for caldata in ['MNIST']:
    # for size in [2000, 500, 200, 50, 20, 5]:
    for metrics in all_metrics:
        print(metrics)
        for size in [200]:
            for mode in ['t']:
                print(mode)
                print(f'-----  Query_size:{size} \t Cal data:{caldata}  -----')
                for k in klist:
                    logname = f'caldata={caldata}_epoch={epoch}_k={k}_calsize=10000'
                    for fold in range(7):
                        queryset = pd.read_csv(f'./saves_new/{expt}/EMA_{querydata}/query_set/binarized_{logname}_fold{fold}.csv')  
                        if fold == 0 or fold == 6:
                            ground_truth = 'Query is not in base'
                        else:
                            ground_truth = 'Query is in base'
                        t, pv = get_member_ratio(queryset[:size], metrics, mode=mode)

                        EMA_res = np.around(pv, decimals=2)
                        if fold == 0:
                            EMA_res_table['SVHN'][f'k={100-k}'] = EMA_res
                        else:
                            EMA_res_table[f'M{fold}'][f'k={100-k}'] = EMA_res

                print(EMA_res_table, '\n')
                EMA_res_table = pd.DataFrame(columns=['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'SVHN'], index=[f'k={100-k}' for k in klist])

['correctness']
t
-----  Query_size:200 	 Cal data:MNIST  -----


FileNotFoundError: [Errno 2] File b'./saves_new/dropout0.1/EMA_MNIST/query_set/binarized_caldata=MNIST_epoch=50_k=10_calsize=10000_fold0.csv' does not exist: b'./saves_new/dropout0.1/EMA_MNIST/query_set/binarized_caldata=MNIST_epoch=50_k=10_calsize=10000_fold0.csv'

In [4]:
### Run EMA on chest X-ray datasets, Table 3.b and Table 7
querydata = 'COVIDx'
expt = "l2"
epoch = 30

EMA_res_table = pd.DataFrame(columns=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CXR'], index=[f'k={100-k}' for k in klist])
for caldata in ['COVIDx']:
    for size in [800, 500, 200, 50, 20, 5]:
        print(f'-----  Query_size:{size} \t Cal data:{caldata}  -----')
        for mode in ['t']:
            print(f'Using {mode}-test')
            for k in klist:
                logname = f'caldata={caldata}_epoch={epoch}_k={k}_calsize=4000'
                for fold in range(7):
                    queryset = pd.read_csv(f'./saves_new/{expt}/EMA_{querydata}/query_set/binarized_{logname}_fold{fold}.csv')  

                    if fold == 0 or fold == 6:
                        ground_truth = 'Query is not in base'
                    else:
                        ground_truth = 'Query is in base'
                    t, pv = get_member_ratio(queryset[:size], skip=['modified entropy'], mode=mode)

                    EMA_res = np.around(pv, decimals=2)
                    if fold == 0:
                        EMA_res_table['CXR'][f'k={100-k}'] = EMA_res
                    else:
                        EMA_res_table[f'C{fold}'][f'k={100-k}'] = EMA_res

            print(EMA_res_table, '\n')
            EMA_res_table = pd.DataFrame(columns=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CXR'], index=[f'k={100-k}' for k in klist])

-----  Query_size:800 	 Cal data:COVIDx  -----
Using t-test
      C1 C2 C3 C4 C5   C6  CXR
k=100  1  1  1  1  1  0.0  0.0
k=90   1  1  1  1  1  0.0  0.0
k=80   1  1  1  1  1  0.0  0.0
k=70   1  1  1  1  1  0.0  0.0
k=60   1  1  1  1  1  0.0  0.0
k=50   1  1  1  1  1  0.0  0.0 

-----  Query_size:500 	 Cal data:COVIDx  -----
Using t-test
      C1 C2 C3 C4 C5   C6  CXR
k=100  1  1  1  1  1  0.0  0.0
k=90   1  1  1  1  1  0.0  0.0
k=80   1  1  1  1  1  0.0  0.0
k=70   1  1  1  1  1  0.0  0.0
k=60   1  1  1  1  1  0.0  0.0
k=50   1  1  1  1  1  0.0  0.0 

-----  Query_size:200 	 Cal data:COVIDx  -----
Using t-test
      C1 C2 C3 C4 C5   C6  CXR
k=100  1  1  1  1  1  0.0  0.0
k=90   1  1  1  1  1  0.0  0.0
k=80   1  1  1  1  1  0.0  0.0
k=70   1  1  1  1  1  0.0  0.0
k=60   1  1  1  1  1  0.0  0.0
k=50   1  1  1  1  1  0.0  0.0 

-----  Query_size:50 	 Cal data:COVIDx  -----
Using t-test
      C1 C2 C3 C4 C5    C6   CXR
k=100  1  1  1  1  1  0.01  0.04
k=90   1  1  1  1  1   0.0   0.0
k=80 